# 深度学习100例-生成对抗网络（GAN）手写数字生成 | 第18天

## 1.前期工作

### 1.1 导入数据

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, datasets, Sequential, Model, optimizers
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D

import matplotlib.pyplot as plt
import numpy             as np
import sys,os,pathlib

### 1.2 定义训练参数

In [8]:
img_shape  = (28, 28, 1)
latent_dim = 200

## 2、什么是[生成对抗网络](https://so.csdn.net/so/search?q=%E7%94%9F%E6%88%90%E5%AF%B9%E6%8A%97%E7%BD%91%E7%BB%9C&spm=1001.2101.3001.7020)

### 2.1 简单介绍

`生成对抗网络（GAN）` 包含生成器和判别器，两个模型通过对抗训练不断学习、进化。

- `生成器(Generator)`：生成数据（大部分情况下是图像），目的是“骗过”判别器。

- `鉴别器(Discriminator)`：判断这张图像是真实的还是机器生成的，目的是找出生成器生成的“假数据”。

### 2.2 应用领域

GAN 的应用十分广泛，它的应用包括图像合成、风格迁移、照片修复以及照片编辑，数据增强等等。

**1）风格迁移**

图像风格迁移是将图像A的风格转换到图像B中去，得到新的图像。

![在这里插入图片描述](https://img-blog.csdnimg.cn/img_convert/1e31ea8f27a4f28a68e481ed346fc423.png#pic_center)

**2）图像生成**

GAN 不但能生成人脸，还能生成其他类型的图片，比如漫画人物。

![](https://img-blog.csdnimg.cn/img_convert/9562698abcb536d2a76527c7265ac6f3.png#pic_center)
    
    
## 3.网络结构

简单来讲，就是用生成器生成手写数字图像，用鉴别器鉴别图像的真假。二者相互对抗学习（卷），在对抗学习（卷）的过程中不断完善自己，直至生成器可以生成以假乱真的图片（鉴别器无法判断其真假）。结构图如下：

![在这里插入图片描述](https://img-blog.csdnimg.cn/img_convert/f75878b04b4e632f27130592a1b4b290.png#pic_center)

**GAN步骤：**

- 1.生成器（Generator）接收随机数并返回生成图像。
- 2.将生成的数字图像与实际数据集中的数字图像一起送到鉴别器（Discriminator）。
- 3.鉴别器（Discriminator）接收真实和假图像并返回概率，0到1之间的数字，1表示真，0表示假。


## 4.构建生成器

In [9]:
def build_generator():
    # ======================================= #
    #     生成器，输入一串随机数字生成图片
    # ======================================= #
    model = Sequential([
        layers.Dense(256, input_dim=latent_dim),
        layers.LeakyReLU(alpha=0.2),               # 高级一点的激活函数
        layers.BatchNormalization(momentum=0.8),   # BN 归一化
        
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        
        layers.Dense(np.prod(img_shape), activation='tanh'),
        layers.Reshape(img_shape)
    ])

    noise = layers.Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## 5.构建鉴别器

In [10]:
def build_discriminator():
    # ===================================== #
    #   鉴别器，对输入的图片进行判别真假
    # ===================================== #
    model = Sequential([
        layers.Flatten(input_shape=img_shape),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])

    img = layers.Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

![在这里插入图片描述](https://img-blog.csdnimg.cn/img_convert/f75878b04b4e632f27130592a1b4b290.png#pic_center)

 -    鉴别器训练原理：通过对输入的图片进行鉴别，从而达到提升的效果
 -    生成器训练原理：通过鉴别器对其生成的图片进行鉴别，来实现提升

In [11]:
# 创建判别器
discriminator = build_discriminator()
# 定义优化器
optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

# 创建生成器 
generator = build_generator()
gan_input = layers.Input(shape=(latent_dim,))
img = generator(gan_input)

# 在训练generate的时候不训练discriminator
discriminator.trainable = False

# 对生成的假图片进行预测
validity = discriminator(img)
combined = Model(gan_input, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

## 6.训练模型

### 6.1 保存样例图片

In [12]:
def sample_images(epoch):
    """
    保存样例图片
    """
    row, col = 4, 4
    noise = np.random.normal(0, 1, (row*col, latent_dim))
    gen_imgs = generator.predict(noise)

    fig, axs = plt.subplots(row, col)
    cnt = 0
    for i in range(row):
        for j in range(col):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%05d.png" % epoch)
    plt.close()

### 6.2训练模型

`train_on_batch`：函数接受单批数据，执行反向传播，然后更新模型参数，该批数据的大小可以是任意的，即，它不需要提供明确的批量大小，属于精细化控制训练模型。

In [13]:
def train(epochs, batch_size=128, sample_interval=50):
    # 加载数据
    (train_images,_), (_,_) = tf.keras.datasets.mnist.load_data()

    # 将图片标准化到 [-1, 1] 区间内   
    train_images = (train_images - 127.5) / 127.5
    # 数据
    train_images = np.expand_dims(train_images, axis=3)

    # 创建标签
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # 进行循环训练
    for epoch in range(epochs): 

        # 随机选择 batch_size 张图片
        idx = np.random.randint(0, train_images.shape[0], batch_size)
        imgs = train_images[idx]      
        
        # 生成噪音
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        # 生成器通过噪音生成图片，gen_imgs的shape为：(128, 28, 28, 1)
        gen_imgs = generator.predict(noise)
        
        # 训练鉴别器 
        d_loss_true = discriminator.train_on_batch(imgs, true)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        # 返回loss值
        d_loss = 0.5 * np.add(d_loss_true, d_loss_fake)

        # 训练生成器
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = combined.train_on_batch(noise, true)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # 保存样例图片
        if epoch % sample_interval == 0:
            sample_images(epoch)

In [15]:
train(epochs=30000, batch_size=256, sample_interval=200)

8/8 [==============================] - 0s 5ms/step
0 [D loss: 0.528410, acc.: 56.05%] [G loss: 0.420333]
8/8 [==============================] - 0s 5ms/step
1 [D loss: 0.444524, acc.: 58.59%] [G loss: 0.417426]
8/8 [==============================] - 0s 4ms/step
2 [D loss: 0.431905, acc.: 57.62%] [G loss: 0.437424]
8/8 [==============================] - 0s 4ms/step
3 [D loss: 0.433804, acc.: 59.18%] [G loss: 0.438122]
8/8 [==============================] - 0s 4ms/step
4 [D loss: 0.430537, acc.: 58.59%] [G loss: 0.456535]
8/8 [==============================] - 0s 4ms/step
5 [D loss: 0.446224, acc.: 57.03%] [G loss: 0.473280]
8/8 [==============================] - 0s 4ms/step
6 [D loss: 0.434001, acc.: 57.62%] [G loss: 0.520286]
8/8 [==============================] - 0s 5ms/step
7 [D loss: 0.417415, acc.: 62.70%] [G loss: 0.546559]
8/8 [==============================] - 0s 6ms/step
8 [D loss: 0.415557, acc.: 64.65%] [G loss: 0.613459]
8/8 [==============================] - 0s 4ms/step
9 [D

8/8 [==============================] - 0s 4ms/step
76 [D loss: 0.025256, acc.: 100.00%] [G loss: 3.388320]
8/8 [==============================] - 0s 4ms/step
77 [D loss: 0.024261, acc.: 100.00%] [G loss: 3.359838]
8/8 [==============================] - 0s 4ms/step
78 [D loss: 0.024629, acc.: 100.00%] [G loss: 3.419208]
8/8 [==============================] - 0s 3ms/step
79 [D loss: 0.024113, acc.: 100.00%] [G loss: 3.457182]
8/8 [==============================] - 0s 4ms/step
80 [D loss: 0.024848, acc.: 100.00%] [G loss: 3.429100]
8/8 [==============================] - 0s 4ms/step
81 [D loss: 0.024301, acc.: 100.00%] [G loss: 3.429625]
8/8 [==============================] - 0s 4ms/step
82 [D loss: 0.024253, acc.: 100.00%] [G loss: 3.473488]
8/8 [==============================] - 0s 5ms/step
83 [D loss: 0.023442, acc.: 100.00%] [G loss: 3.460455]
8/8 [==============================] - 0s 4ms/step
84 [D loss: 0.026573, acc.: 100.00%] [G loss: 3.463246]
8/8 [==============================] 

8/8 [==============================] - 0s 4ms/step
152 [D loss: 0.021890, acc.: 100.00%] [G loss: 4.004826]
8/8 [==============================] - 0s 4ms/step
153 [D loss: 0.023007, acc.: 100.00%] [G loss: 3.988445]
8/8 [==============================] - 0s 4ms/step
154 [D loss: 0.022553, acc.: 100.00%] [G loss: 3.922933]
8/8 [==============================] - 0s 4ms/step
155 [D loss: 0.025074, acc.: 100.00%] [G loss: 3.978951]
8/8 [==============================] - 0s 3ms/step
156 [D loss: 0.024841, acc.: 100.00%] [G loss: 3.938398]
8/8 [==============================] - 0s 4ms/step
157 [D loss: 0.021849, acc.: 100.00%] [G loss: 4.009294]
8/8 [==============================] - 0s 3ms/step
158 [D loss: 0.024811, acc.: 100.00%] [G loss: 4.004487]
8/8 [==============================] - 0s 4ms/step
159 [D loss: 0.026510, acc.: 100.00%] [G loss: 4.033868]
8/8 [==============================] - 0s 4ms/step
160 [D loss: 0.026125, acc.: 100.00%] [G loss: 3.980434]
8/8 [=======================

8/8 [==============================] - 0s 4ms/step
228 [D loss: 0.097674, acc.: 96.88%] [G loss: 3.501182]
8/8 [==============================] - 0s 3ms/step
229 [D loss: 0.068046, acc.: 98.44%] [G loss: 3.435084]
8/8 [==============================] - 0s 3ms/step
230 [D loss: 0.078501, acc.: 98.24%] [G loss: 3.487125]
8/8 [==============================] - 0s 4ms/step
231 [D loss: 0.080539, acc.: 98.24%] [G loss: 3.498977]
8/8 [==============================] - 0s 3ms/step
232 [D loss: 0.074749, acc.: 98.44%] [G loss: 3.584657]
8/8 [==============================] - 0s 3ms/step
233 [D loss: 0.088230, acc.: 97.66%] [G loss: 3.612533]
8/8 [==============================] - 0s 3ms/step
234 [D loss: 0.069157, acc.: 99.22%] [G loss: 3.566456]
8/8 [==============================] - 0s 4ms/step
235 [D loss: 0.075874, acc.: 98.24%] [G loss: 3.571567]
8/8 [==============================] - 0s 4ms/step
236 [D loss: 0.080077, acc.: 98.05%] [G loss: 3.704684]
8/8 [==============================] 

8/8 [==============================] - 0s 4ms/step
305 [D loss: 0.036065, acc.: 99.61%] [G loss: 4.539643]
8/8 [==============================] - 0s 3ms/step
306 [D loss: 0.035569, acc.: 100.00%] [G loss: 4.563880]
8/8 [==============================] - 0s 4ms/step
307 [D loss: 0.033593, acc.: 99.80%] [G loss: 4.602602]
8/8 [==============================] - 0s 4ms/step
308 [D loss: 0.027221, acc.: 100.00%] [G loss: 4.602202]
8/8 [==============================] - 0s 5ms/step
309 [D loss: 0.044142, acc.: 99.80%] [G loss: 4.498907]
8/8 [==============================] - 0s 4ms/step
310 [D loss: 0.035488, acc.: 99.61%] [G loss: 4.616594]
8/8 [==============================] - 0s 5ms/step
311 [D loss: 0.032650, acc.: 99.80%] [G loss: 4.697883]
8/8 [==============================] - 0s 4ms/step
312 [D loss: 0.027127, acc.: 99.80%] [G loss: 4.853010]
8/8 [==============================] - 0s 4ms/step
313 [D loss: 0.030886, acc.: 99.80%] [G loss: 4.806653]
8/8 [==============================

...

8/8 [==============================] - 0s 4ms/step
29643 [D loss: 0.687818, acc.: 55.86%] [G loss: 0.718191]
8/8 [==============================] - 0s 4ms/step
29644 [D loss: 0.678798, acc.: 55.47%] [G loss: 0.715110]
8/8 [==============================] - 0s 4ms/step
29645 [D loss: 0.684411, acc.: 56.25%] [G loss: 0.713404]
8/8 [==============================] - 0s 4ms/step
29646 [D loss: 0.688726, acc.: 55.08%] [G loss: 0.713330]
8/8 [==============================] - 0s 4ms/step
29647 [D loss: 0.681476, acc.: 55.66%] [G loss: 0.720842]
8/8 [==============================] - 0s 4ms/step
29648 [D loss: 0.690423, acc.: 51.95%] [G loss: 0.725873]
8/8 [==============================] - 0s 4ms/step
29649 [D loss: 0.680030, acc.: 54.88%] [G loss: 0.724244]
8/8 [==============================] - 0s 4ms/step
29650 [D loss: 0.689936, acc.: 53.52%] [G loss: 0.729110]
8/8 [==============================] - 0s 4ms/step
29651 [D loss: 0.692444, acc.: 52.15%] [G loss: 0.725430]
8/8 [==============

8/8 [==============================] - 0s 4ms/step
29718 [D loss: 0.678335, acc.: 58.01%] [G loss: 0.739467]
8/8 [==============================] - 0s 4ms/step
29719 [D loss: 0.683543, acc.: 55.86%] [G loss: 0.724010]
8/8 [==============================] - 0s 4ms/step
29720 [D loss: 0.687259, acc.: 53.71%] [G loss: 0.720062]
8/8 [==============================] - 0s 4ms/step
29721 [D loss: 0.690704, acc.: 52.15%] [G loss: 0.729850]
8/8 [==============================] - 0s 4ms/step
29722 [D loss: 0.674554, acc.: 58.59%] [G loss: 0.728849]
8/8 [==============================] - 0s 4ms/step
29723 [D loss: 0.685115, acc.: 53.32%] [G loss: 0.730690]
8/8 [==============================] - 0s 4ms/step
29724 [D loss: 0.689025, acc.: 54.69%] [G loss: 0.732466]
8/8 [==============================] - 0s 5ms/step
29725 [D loss: 0.683927, acc.: 55.47%] [G loss: 0.735542]
8/8 [==============================] - 0s 5ms/step
29726 [D loss: 0.688054, acc.: 52.93%] [G loss: 0.726023]
8/8 [==============

8/8 [==============================] - 0s 4ms/step
29793 [D loss: 0.674044, acc.: 58.98%] [G loss: 0.735387]
8/8 [==============================] - 0s 4ms/step
29794 [D loss: 0.682469, acc.: 58.59%] [G loss: 0.737676]
8/8 [==============================] - 0s 4ms/step
29795 [D loss: 0.676087, acc.: 58.01%] [G loss: 0.730454]
8/8 [==============================] - 0s 4ms/step
29796 [D loss: 0.681664, acc.: 57.42%] [G loss: 0.740948]
8/8 [==============================] - 0s 5ms/step
29797 [D loss: 0.675157, acc.: 59.38%] [G loss: 0.725614]
8/8 [==============================] - 0s 4ms/step
29798 [D loss: 0.689299, acc.: 55.86%] [G loss: 0.738112]
8/8 [==============================] - 0s 4ms/step
29799 [D loss: 0.680344, acc.: 53.52%] [G loss: 0.730879]
8/8 [==============================] - 0s 4ms/step
29800 [D loss: 0.678709, acc.: 55.66%] [G loss: 0.721988]
8/8 [==============================] - 0s 4ms/step
29801 [D loss: 0.676623, acc.: 57.23%] [G loss: 0.719702]
8/8 [==============

8/8 [==============================] - 0s 4ms/step
29868 [D loss: 0.690386, acc.: 52.93%] [G loss: 0.737294]
8/8 [==============================] - 0s 4ms/step
29869 [D loss: 0.692959, acc.: 54.30%] [G loss: 0.744350]
8/8 [==============================] - 0s 4ms/step
29870 [D loss: 0.692306, acc.: 52.34%] [G loss: 0.729108]
8/8 [==============================] - 0s 4ms/step
29871 [D loss: 0.692006, acc.: 55.08%] [G loss: 0.740402]
8/8 [==============================] - 0s 4ms/step
29872 [D loss: 0.692750, acc.: 51.37%] [G loss: 0.734212]
8/8 [==============================] - 0s 4ms/step
29873 [D loss: 0.691229, acc.: 52.54%] [G loss: 0.730469]
8/8 [==============================] - 0s 4ms/step
29874 [D loss: 0.689456, acc.: 53.91%] [G loss: 0.730102]
8/8 [==============================] - 0s 4ms/step
29875 [D loss: 0.696981, acc.: 51.37%] [G loss: 0.738343]
8/8 [==============================] - 0s 4ms/step
29876 [D loss: 0.695155, acc.: 51.76%] [G loss: 0.722307]
8/8 [==============

8/8 [==============================] - 0s 5ms/step
29943 [D loss: 0.694612, acc.: 55.27%] [G loss: 0.720991]
8/8 [==============================] - 0s 4ms/step
29944 [D loss: 0.687123, acc.: 54.10%] [G loss: 0.722169]
8/8 [==============================] - 0s 4ms/step
29945 [D loss: 0.683622, acc.: 57.81%] [G loss: 0.732440]
8/8 [==============================] - 0s 4ms/step
29946 [D loss: 0.686557, acc.: 57.62%] [G loss: 0.729041]
8/8 [==============================] - 0s 4ms/step
29947 [D loss: 0.697259, acc.: 51.76%] [G loss: 0.705520]
8/8 [==============================] - 0s 4ms/step
29948 [D loss: 0.684570, acc.: 56.25%] [G loss: 0.704775]
8/8 [==============================] - 0s 4ms/step
29949 [D loss: 0.695215, acc.: 51.56%] [G loss: 0.707202]
8/8 [==============================] - 0s 5ms/step
29950 [D loss: 0.679154, acc.: 56.64%] [G loss: 0.702808]
8/8 [==============================] - 0s 4ms/step
29951 [D loss: 0.678287, acc.: 58.20%] [G loss: 0.704584]
8/8 [==============

## 7.生成动图

如果报错：`ModuleNotFoundError: No module named 'imageio'` 可以使用：`pip install imageio` 安装 `imageio` 库。

In [18]:
import imageio

def compose_gif():
    # 图片地址
    data_dir = r"C:\Users\fangjian\images"
    data_dir = pathlib.Path(data_dir)
    paths    = list(data_dir.glob('*'))
    
    gif_images = []
    for path in paths:
        print(path)
        gif_images.append(imageio.imread(path))
    imageio.mimsave("test.gif",gif_images,fps=2)
    
compose_gif()

C:\Users\fangjian\images\00000.png
C:\Users\fangjian\images\00200.png
C:\Users\fangjian\images\00400.png
C:\Users\fangjian\images\00600.png
C:\Users\fangjian\images\00800.png
C:\Users\fangjian\images\01000.png
C:\Users\fangjian\images\01200.png
C:\Users\fangjian\images\01400.png
C:\Users\fangjian\images\01600.png
C:\Users\fangjian\images\01800.png
C:\Users\fangjian\images\02000.png
C:\Users\fangjian\images\02200.png
C:\Users\fangjian\images\02400.png
C:\Users\fangjian\images\02600.png
C:\Users\fangjian\images\02800.png
C:\Users\fangjian\images\03000.png
C:\Users\fangjian\images\03200.png
C:\Users\fangjian\images\03400.png
C:\Users\fangjian\images\03600.png
C:\Users\fangjian\images\03800.png
C:\Users\fangjian\images\04000.png
C:\Users\fangjian\images\04200.png
C:\Users\fangjian\images\04400.png
C:\Users\fangjian\images\04600.png
C:\Users\fangjian\images\04800.png
C:\Users\fangjian\images\05000.png
C:\Users\fangjian\images\05200.png
C:\Users\fangjian\images\05400.png
C:\Users\fangjian\im

C:\Users\fangjian\AppData\Local\Temp\ipykernel_6396\1096145853.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  gif_images.append(imageio.imread(path))


C:\Users\fangjian\images\08400.png
C:\Users\fangjian\images\08600.png
C:\Users\fangjian\images\08800.png
C:\Users\fangjian\images\09000.png
C:\Users\fangjian\images\09200.png
C:\Users\fangjian\images\09400.png
C:\Users\fangjian\images\09600.png
C:\Users\fangjian\images\09800.png
C:\Users\fangjian\images\10000.png
C:\Users\fangjian\images\10200.png
C:\Users\fangjian\images\10400.png
C:\Users\fangjian\images\10600.png
C:\Users\fangjian\images\10800.png
C:\Users\fangjian\images\11000.png
C:\Users\fangjian\images\11200.png
C:\Users\fangjian\images\11400.png
C:\Users\fangjian\images\11600.png
C:\Users\fangjian\images\11800.png
C:\Users\fangjian\images\12000.png
C:\Users\fangjian\images\12200.png
C:\Users\fangjian\images\12400.png
C:\Users\fangjian\images\12600.png
C:\Users\fangjian\images\12800.png
C:\Users\fangjian\images\13000.png
C:\Users\fangjian\images\13200.png
C:\Users\fangjian\images\13400.png
C:\Users\fangjian\images\13600.png
C:\Users\fangjian\images\13800.png
C:\Users\fangjian\im